# Основные операторы PostgreSQL.

## Цель
Научиться группировать и агрегировать данные, считать на их основе множество показателей.

## Формулировка задания

Дано два csv-файла с данными о клиентах (customer.csv) и их транзакциях (transaction.csv).
Необходимо выполнить следующее:

1. (1 балл) Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.
2. (1 балл) Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.
3. (1 балл) Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT.
4. (1 балл) Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций.
5. (2 балла) Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества транзакций. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат.
6. (2 балла) Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях).
7. (2 балла) Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы.

In [1]:
import secretstorage as sec

def get_database_password(connection: sec.DBusConnection) -> str:
    collection = sec.get_default_collection(connection)
    # Поиск секрета по атрибуту "service"
    for item in collection.search_items({'service': 'ipynb'}):
        if item.is_locked():
            item.unlock()
        return item.get_secret().decode("cp1251").strip()

    return ""


In [2]:
connection = sec.dbus_init()

In [11]:
import pandas as pd
import psycopg2 as pg
import numpy as np

# Connect to database
db_conn = pg.connect(dbname="study", host="localhost", user="postgres", password=get_database_password(connection))

In [29]:
customer_count = pd.read_sql_query("SELECT COUNT(*) FROM hw_3.customer", db_conn)["count"].values[0]
customer_count

/tmp/ipykernel_56243/3011983746.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customer_count = pd.read_sql_query("SELECT COUNT(*) FROM hw_3.customer", db_conn)["count"].values[0]


np.int64(4000)

### Задание 1

(1 балл) Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.

In [31]:
customer_count_by_category_query = """
SELECT job_industry_category, COUNT(*) FROM hw_3.customer
GROUP BY job_industry_category
ORDER BY count DESC
"""
data = pd.read_sql_query(customer_count_by_category_query, db_conn)
display(data)

assert(np.sum(data["count"]) == customer_count)

/tmp/ipykernel_56243/2601018405.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql_query(customer_count_by_category_query, db_conn)


,job_industry_category,count
0,Manufacturing,799
1,Financial Services,774
2,n/a,656
3,Health,602
4,Retail,358
5,Property,267
6,IT,223
7,Entertainment,136
8,Argiculture,113
9,Telecommunications,72


### Задание 2

(1 балл) Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности.

In [78]:
transaction_per_month_by_category = """
WITH CustomerAndTransaction AS (
    SELECT * FROM hw_3.transaction AS t
    LEFT JOIN hw_3.customer AS c
    ON c.customer_id = t.customer_id
)

SELECT job_industry_category, EXTRACT(MONTH FROM TO_DATE(transaction_date, 'DD.MM.YYYY')) as month_number, SUM(list_price) as total_price
FROM CustomerAndTransaction
GROUP BY job_industry_category, month_number
ORDER BY month_number, job_industry_category
"""

pd.read_sql_query(transaction_per_month_by_category, db_conn)

/tmp/ipykernel_56243/3460571888.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(transaction_per_month_by_category, db_conn)


,job_industry_category,month_number,total_price
0,Argiculture,1.0,3918068.0
1,Entertainment,1.0,5212721.0
2,Financial Services,1.0,30309572.0
3,Health,1.0,25680500.0
4,IT,1.0,8821143.0
...,...,...,...
117,Manufacturing,12.0,26871772.0
118,n/a,12.0,26883048.0
119,Property,12.0,9892813.0
120,Retail,12.0,13013370.0


### Задание 3

(1 балл) Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT.

In [91]:
online_arroved_transaction_by_brand = """
WITH CustomerAndTransaction AS (
    SELECT t.brand, t.online_order, t.order_status, c.job_industry_category FROM hw_3.transaction AS t
    LEFT JOIN hw_3.customer AS c
    ON c.customer_id = t.customer_id
)

SELECT brand, COUNT(*) AS order_count FROM CustomerAndTransaction
WHERE online_order=TRUE AND order_status='Approved' AND job_industry_category='IT'
GROUP BY brand
"""
pd.read_sql_query(online_arroved_transaction_by_brand, db_conn)

/tmp/ipykernel_56243/1976679137.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(online_arroved_transaction_by_brand, db_conn)


,brand,order_count
0,,8
1,Trek Bicycles,82
2,WeareA2B,90
3,Solex,101
4,Giant Bicycles,89
5,OHM Cycles,78
6,Norco Bicycles,92


### Задание 4

(2 балла) Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества транзакций. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат.

In [111]:
client_info_group_by = """
WITH CustomerAndTransaction AS (
    SELECT c.customer_id, t.transaction_id, t.list_price FROM hw_3.customer AS c
    LEFT JOIN hw_3.transaction AS t
    ON c.customer_id = t.customer_id
)

SELECT
    customer_id, COALESCE(SUM(list_price), 0) AS total_sum,
    COALESCE(MAX(list_price), 0) as max_price,
    COALESCE(MIN(list_price), 0) as min_price,
    COALESCE(COUNT(transaction_id), 0) AS transaction_count
FROM CustomerAndTransaction
GROUP BY customer_id
ORDER BY total_sum DESC, transaction_count DESC
"""
display(pd.read_sql_query(client_info_group_by, db_conn))

client_info_window_functions = """
WITH CustomerAndTransaction AS (
    SELECT c.customer_id, t.transaction_id, t.list_price FROM hw_3.customer AS c
    LEFT JOIN hw_3.transaction AS t
    ON c.customer_id = t.customer_id
)

SELECT 
    customer_id,
    COALESCE(SUM(list_price) OVER(by_customer), 0) AS total_sum,
    COALESCE(MAX(list_price) OVER(by_customer), 0) as max_price,
    COALESCE(MIN(list_price) OVER(by_customer), 0) as min_price,
    COALESCE(COUNT(transaction_id) OVER(by_customer), 0) AS transaction_count 
FROM CustomerAndTransaction
WINDOW by_customer AS (PARTITION BY customer_id)
ORDER BY total_sum DESC, transaction_count DESC

"""
pd.read_sql_query(client_info_window_functions, db_conn)


/tmp/ipykernel_56243/2330269627.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(client_info_group_by, db_conn))


,customer_id,total_sum,max_price,min_price,transaction_count
0,941,1789846.0,209147.0,105751.0,10
1,1887,1713393.0,209147.0,68863.0,11
2,1129,1676050.0,199293.0,17653.0,13
3,2788,1565892.0,208394.0,17778.0,11
4,1302,1548720.0,197736.0,7116.0,13
...,...,...,...,...,...
3995,3686,0.0,0.0,0.0,0
3996,3852,0.0,0.0,0.0,0
3997,3679,0.0,0.0,0.0,0
3998,3816,0.0,0.0,0.0,0


/tmp/ipykernel_56243/2330269627.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(client_info_window_functions, db_conn)


,customer_id,total_sum,max_price,min_price,transaction_count
0,941,1789846.0,209147.0,105751.0,10
1,941,1789846.0,209147.0,105751.0,10
2,941,1789846.0,209147.0,105751.0,10
3,941,1789846.0,209147.0,105751.0,10
4,941,1789846.0,209147.0,105751.0,10
...,...,...,...,...,...
20499,3800,0.0,0.0,0.0,0
20500,3801,0.0,0.0,0.0,0
20501,3802,0.0,0.0,0.0,0
20502,3803,0.0,0.0,0.0,0


### Сравнение результатов
Результат **GROP BY** и **WINDOW** схож, за исключением дублирования, которые пораждается оконная функция(их можно убрать использую DISTINCT). Оконные функции позволяют анализировать несгруппированные данные, а также лучше подходит для аналитических задач.


### Задание 5

(2 балла) Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы

In [188]:
client_min_transactions = """
WITH CustomerTotalPrice AS (
    SELECT customer_id, SUM(list_price) as total_price FROM hw_3.transaction
    GROUP BY customer_id
),
CustomerMinPrice AS (
    SELECT MIN(total_price) as min_price FROM CustomerTotalPrice
)

SELECT
    t.customer_id, c.first_name, c.last_name, t.total_price
FROM CustomerTotalPrice AS t
JOIN hw_3.customer AS c on c.customer_id = t.customer_id
WHERE t.total_price = (SELECT min_price FROM CustomerMinPrice)
"""
display(pd.read_sql_query(client_min_transactions, db_conn))

client_max_transactions = """
WITH CustomerTotalPrice AS (
    SELECT customer_id, SUM(list_price) as total_price FROM hw_3.transaction
    GROUP BY customer_id
),
CustomerMaxPrice AS (
    SELECT MAX(total_price) as max_price FROM CustomerTotalPrice
)

SELECT
    t.customer_id, c.first_name, c.last_name, t.total_price
FROM CustomerTotalPrice AS t
JOIN hw_3.customer AS c on c.customer_id = t.customer_id
WHERE t.total_price = (SELECT max_price FROM CustomerMaxPrice)
"""
display(pd.read_sql_query(client_max_transactions, db_conn))



/tmp/ipykernel_56243/4039698589.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(client_min_transactions, db_conn))


,customer_id,first_name,last_name,total_price
0,1529,Tansy,Beltzner,1810.0


/tmp/ipykernel_56243/4039698589.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  display(pd.read_sql_query(client_max_transactions, db_conn))


,customer_id,first_name,last_name,total_price
0,941,Tye,Doohan,1789846.0


### Задание 6

(1 балл) Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций

In [215]:
first_clients_transaction = """
WITH ranked_transaction AS (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY TO_DATE(transaction_date, 'DD.MM.YYYY')) as rk
    FROM hw_3.transaction
)

SELECT * FROM ranked_transaction WHERE rk = 1
"""
pd.read_sql_query(first_clients_transaction, db_conn)

/tmp/ipykernel_56243/1490108170.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(first_clients_transaction, db_conn)


,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,rk
0,9785,72,1,05.01.2017,False,Approved,Norco Bicycles,Standard,medium,medium,3604.0,27030.0,1
1,2261,1,2,04.05.2017,True,Approved,Giant Bicycles,Standard,medium,medium,14035.0,95482.0,1
2,10302,33,3,23.02.2017,False,Approved,Giant Bicycles,Standard,medium,small,131144.0,116718.0,1
3,12441,95,4,03.04.2017,False,Approved,Giant Bicycles,Standard,medium,large,56956.0,52843.0,1
4,2291,23,5,03.03.2017,True,Approved,Norco Bicycles,Mountain,low,small,68863.0,61288.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3489,8276,18,3497,01.09.2017,True,Approved,Solex,Standard,medium,medium,57527.0,43145.0,1
3490,13469,12,3498,03.02.2017,True,Approved,WeareA2B,Standard,medium,medium,123115.0,16160.0,1
3491,2794,62,3499,12.01.2017,False,Approved,Solex,Standard,medium,medium,47816.0,29872.0,1
3492,6309,69,3500,09.01.2017,True,Approved,Giant Bicycles,Road,medium,medium,7929.0,59468.0,1


### Задание 7

(2 балла) Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях)

In [253]:
transaction_date_distance = """
WITH diff_transaction AS (
    SELECT *,
    TO_DATE(LAG(transaction_date) OVER(by_customer_sorted_by_date), 'DD.MM.YYYY') as prev_date,
    TO_DATE(transaction_date, 'DD.MM.YYYY') -
        TO_DATE(LAG(transaction_date) OVER(by_customer_sorted_by_date), 'DD.MM.YYYY') as diff
    FROM hw_3.transaction
    WINDOW by_customer_sorted_by_date AS (PARTITION BY customer_id ORDER BY TO_DATE(transaction_date, 'DD.MM.YYYY'))
),
ranked_transaction AS (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY diff DESC) as rk
    FROM diff_transaction
    WHERE diff IS NOT NULL
)

SELECT c.first_name, c.last_name, c.job_title, diff as transaction_diff FROM ranked_transaction as t
JOIN hw_3.customer as c
    ON c.customer_id = t.customer_id
WHERE rk = 1
"""
pd.read_sql_query(transaction_date_distance, db_conn)

/tmp/ipykernel_56243/3427531634.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(transaction_date_distance, db_conn)


,first_name,last_name,job_title,transaction_diff
0,Laraine,Medendorp,Executive Secretary,188
1,Eli,Bockman,Administrative Officer,74
2,Arlin,Dearle,Recruiting Manager,70
3,Talbot,,,76
4,Sheila-kathryn,Calton,Senior Editor,120
...,...,...,...,...
3439,Danya,Burnyeat,Editor,26
3440,Thia,O'Day,Administrative Assistant IV,57
3441,Lois,Abrahim,,106
3442,Shelton,Tewkesberrie,,140


In [25]:
# Close connection to the database.
db_conn.close()